In [1]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import statsmodels.stats.multitest as smm
from scipy import stats
import seaborn as sns

from Bio import Seq

pd.options.mode.chained_assignment = None

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

Change the following parameters to your output directory (where you want output files to be written), your metafile, and TNseq output. 

If these files aren't in the same directory as this notebook, you need use the full path. For example, the full path of 'barseq_output' would be '/usr2/people/clairedubin/barseq/barseq_output'

Also change the control and experimental temperatures if needed.

In [2]:
#where the BarSeq FASTQs are
FASTQ_directory = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads'


## Barseq metafile

You'll need to create your own metafile - easiest way is edit in Excel. A sample metafile is located at: /usr2/people/clairedubin/barseq/Kluyv_BarSeq_metadata.txt

    Fastq: path to barseq FASTQ for each competition
    SampleName: unique name for each competition, MUST include ctrl_temp or exp_temp as specified above
    DualIndex: index from sequencing (in read name lines of FASTQ and the sequencing info file that Adam sends)
    
    UsePrecounted: True if you've already run the Barseq counting script on a competition, False otherwise
    
    poolfile: path to Tnseq output file, should be the same for each competition
    output_dir: path to output directory
    minRandom: number of bases between start of read and DualIndex, usually in sequence of 1,2,3,4,1,2,3,4 for each sample
    maxRandom: same as minRandom
    
    BeforeBarcode: sequence preceding all barcodes
    AfterBarcode: sequence following all barcodes
    BarcodeLengths: lengths of barcodes to search for, with expected length of barcode first

    

In [3]:
!ls {FASTQ_directory}/*fastq

/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37A1_S1_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37A2_S2_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37B1_S3_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37B2_S4_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50A1_S5_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq

In [4]:
#format: sample_name, fastq_path, dual_index, offset
#add or delete more lines as needed

sample_info = [('50_A2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50A2_S6_L001_R1_001.fastq', 'CAGATC', '3')
               ]

In [5]:
!head /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37A1_S1_L001_R1_001.fastq

@VH00771:102:AAGC27TM5:1:1101:63373:1038 1:N:0:ATTACG
CNCACTAGTCGACCTGCAGCGTACGGTGCAACAAACAGTATGCGCAGAGACCTCGTGGACATCAGATCGGAAGAGCACATGTTTGGATTCTAGTCACATTACGATTTGGGGGGGGGGGGGGGGGGGGGGGGG
+
I#IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII-II9IIIIIIIIII-III99I-9--9I-9----999I99-IIIII
@VH00771:102:AAGC27TM5:1:1101:63449:1038 1:N:0:ATTACG
GNCACTAGTCGACCTGCAGCGTACGTAGGCAACCACACGAGCCGTAGAGACCTCGTGGACATCAGATCGGAAGAGCACATGTTTGGACTCTAGTCATATTACGATTTGGGGGGGGGGGGGGGGGGGGGGGGG
+
I#IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII-IIIIIIIIIIIIIIIIIIIIIIII9II9IIIII-IIII9II99-9I99-99------9--II-9I9III
@VH00771:102:AAGC27TM5:1:1101:63525:1038 1:N:0:ATTACG
TNCACTAGTCGACCTGCAGCGTACGATACGAAATTATGAGCTGTCAGAGACCTCGTGGACATCAGATCGGAAGAGCACATGTTTGGACTCTAGTCACATTACGATTTGGGGGGGGGGGGGGGGGGGGGGGGG


### To convert the annotated file to csv and not to be separated by tab (execute the next three lines only if necessary)

In [6]:
poolfile1 = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_13k_mutants_mapping_output_all_fastqs_available/Tnseq_13k_mutants_all_digestion_time_poolfile_annotated.csv'
df=pd.read_csv(poolfile1,sep='\t')

In [7]:
df.to_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_13k_mutants_mapping_output_all_fastqs_available/Tnseq_13k_mutants_all_digestion_time_poolfile_annotated_1.csv',index=None)

In [8]:
df=pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_13k_mutants_mapping_output_all_fastqs_available/Tnseq_13k_mutants_all_digestion_time_poolfile_annotated_1.csv')

In [9]:
#these variables should be the same for each replicate

metafile_name = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/no_mapping/BarSeq_metafile_13k_mutants_no_mapping.txt'
logfile_name = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/no_mapping/BarSeq_metafile_13k_mutants_no_mapping.log'
poolfile = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_13k_mutants_mapping_output_all_fastqs_available/Tnseq_13k_mutants_all_digestion_time_poolfile_annotated_1.csv'
output_dir = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/no_mapping'
before_barcode = 'GTCGACCTGCAGCGTACG' #bottom strand sequence was used
#before_barcode = 'GATGTCCACGAGGTCTCT' #what was there before 
after_barcode =  'AGAGACCTCGTGGACATC' #bottom strand sequence was used
#after_barcode = 'CGTACGCTGCAGGTCGAC'  #what was there before 
barcode_lengths = '20,19,18,21'

In [10]:
#if there are any samples where you already counted barcodes, add the sample names here
precounted_samples = []

In [11]:
metafile_columns = ['FileIndex','SampleName', 'Fastq', 'DualIndex', 'Poolfile', 'OutputDir', 'minRandom', 'maxRandom',
                   'BeforeBarcode', 'AfterBarcode', 'BarcodeLengths', 'UsePrecounted']

with open(metafile_name, 'w') as f:
    
    #write column names
    f.write('\t'.join(metafile_columns)+'\n')
    
    #write a line for each pool
    for sample_name, fastq, dual_index, offset in sample_info:
        
        #DUAL INDECES ARE BACKWARDS FROM WHAT THE FLOWCELL OUTPUT SAYS
        to_write = '\t'.join([sample_name, sample_name, fastq, dual_index[::-1], poolfile, output_dir, offset, offset,
                             before_barcode, after_barcode, barcode_lengths, 'False'])
        f.write(to_write+'\n')
        
    for precounted_sample_name in precounted_samples:
        
        to_write = '\t'.join([sample_name, sample_name, fastq, dual_index, poolfile, output_dir, offset, offset,
                             before_barcode, after_barcode, barcode_lengths, 'True'])




### Check metafile

In [12]:
pd.read_csv(metafile_name, sep='\t')

,FileIndex,SampleName,Fastq,DualIndex,Poolfile,OutputDir,minRandom,maxRandom,BeforeBarcode,AfterBarcode,BarcodeLengths,UsePrecounted
0,50_A2,50_A2,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50A2_S6_L001_R1_001.fastq,CTAGAC,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_13k_mutants_mapping_output_all_fastqs_available/Tnseq_13k_mutants_all_digestion_time_poolfile_annotated_1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/no_mapping,3,3,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False


## Barseq counting

Might take a few hours!

In [14]:
!python3 /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/no_mapping/RBseq_Count_BarCodes_without_tnseq_poolfile.py -m {metafile_name} -l {logfile_name}

['TTCG', 'CTCG', 'GTCG', 'AACG', 'ACCG', 'AGCG', 'ATAG', 'ATTG', 'ATGG', 'ATCA', 'ATCT', 'ATCC']
  Counting barcodes in /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50A2_S6_L001_R1_001.fastq
  Estimated sequencing error rate for barcodes: 0.5%
  Barcodes seen once: 54058189, twice: 1175829, three times or more: 1721224
  Chao estimate of population size: 1242649993.3067312
Saving summary statistics for fastqs to: /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/no_mapping/fastqSummaryStats.txt


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'KFRB0029_37A1'
y_column = 'KFRB0029_37B1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('KFRB0029_37A Abundance', fontsize=11)
plt.ylabel('KFRB0029_37B Abundance', fontsize=11)
plt.title('Plot of TN_insertion abundances at 37 oC (Bio reps)', fontsize=13, color='purple')
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("fitness_of_TN_abundances_01_03_2024_37oC.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'KFRB0029_37A1'
y_column = 'KFRB0029_37B1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('KFRB0029_37A Abundance', fontsize=11)
plt.ylabel('KFRB0029_37B Abundance', fontsize=11)
plt.title('Plot of TN_insertion abundances at 37 oC (Bio reps)', fontsize=13, color='purple')

# Adjust the x and y-axis scale ranges
plt.xlim(0, 3000)  # Adjust the range for the x-axis
plt.ylim(0, 3000)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)

#plt.savefig("fitness_of_TN_abundances_03_01_2024_37oC_1a.png", dpi=600, bbox_inches='tight')
plt.show()


## to normalize the Barseq pool count file based on the number of reads obtained

### this is the normalization that worked with the right result

In [ ]:
import pandas as pd

# Load the count file into a DataFrame
count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_new_37•C_with_older_50•C_1.txt'
df = pd.read_csv(count_file_path, sep='\t')

# Calculate the total counts for each replicate
df['Total_Count_37•C (A)'] = df['37•C (A)'].sum()
df['Total_Count_37•C (B)'] = df['37•C (B)'].sum()
df['Total_Count_45•C (A)'] = df['45•C (A)'].sum()
df['Total_Count_45•C (B)'] = df['45•C (B)'].sum()
df['Total_Count_45•C (C)'] = df['45•C (C)'].sum()
df['Total_Count_45•C (D)'] = df['45•C (D)'].sum()
df['Total_Count_50•C (A)'] = df['50•C (A)'].sum()
df['Total_Count_50•C (B)'] = df['50•C (B)'].sum()
df['Total_Count_50•C (C)'] = df['50•C (C)'].sum()
df['Total_Count_50•C (D)'] = df['50•C (D)'].sum()


# Normalize counts for each replicate by dividing by its own Total_Count
df['normalized_37•C (A)'] = df['37•C (A)'] / df['Total_Count_37•C (A)']
df['normalized_37•C (B)'] = df['37•C (B)'] / df['Total_Count_37•C (B)']
df['normalized_45•C (A)'] = df['45•C (A)'] / df['Total_Count_45•C (A)']
df['normalized_45•C (B)'] = df['45•C (B)'] / df['Total_Count_45•C (B)']
df['normalized_45•C (C)'] = df['45•C (C)'] / df['Total_Count_45•C (C)']
df['normalized_45•C (D)'] = df['45•C (D)'] / df['Total_Count_45•C (D)']
df['normalized_50•C (A)'] = df['50•C (A)'] / df['Total_Count_50•C (A)']
df['normalized_50•C (B)'] = df['50•C (B)'] / df['Total_Count_50•C (B)']
df['normalized_50•C (C)'] = df['50•C (C)'] / df['Total_Count_50•C (C)']
df['normalized_50•C (D)'] = df['50•C (D)'] / df['Total_Count_50•C (D)']

# Save the normalized count file
normalized_count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_biorep_all_temp_normalized.csv'
df.to_csv(normalized_count_file_path, index=False)


##### To make the fitness abundance agaian using the normalized values

In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_biorep_all_temp_normalized.csv'
df = pd.read_csv(csv_file_path)

# Save the DataFrame to a text file (txt)
txt_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_biorep_all_temp_normalized.txt'
df.to_csv(txt_file_path, sep='\t', index=False)


In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_biorep_all_temp_normalized.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'normalized_37•C (A)'
y_column = 'normalized_37•C (B)'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('normalized_37•C (A) Abundance', fontsize=11)
plt.ylabel('Normalized_37•C (B) Abundance', fontsize=11)
plt.title('Plot of TN_insertion abundances_37 oC_normalized (A)', fontsize=13, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.04)  # Adjust the range for the x-axis
plt.ylim(0, 0.04)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_37oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_biorep_all_temp_normalized.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'normalized_37•C (A)'
y_column = 'normalized_37•C (B)'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('normalized_37•C (A) Abundance', fontsize=11)
plt.ylabel('Normalized_37•C (B) Abundance', fontsize=11)
plt.title('Plot of TN_insertion abundances_37 oC_normalized (A)', fontsize=13, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.015)  # Adjust the range for the x-axis
plt.ylim(0, 0.015)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_37oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_06_12_2024/normalized_combined_fastqs_barseq_pool_counts_06_12_2024.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'KFRB021_1'
y_column = 'KFRB022_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_KFRB021_1 Abundance', fontsize=14)
plt.ylabel('Normalized_KFRB022_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances_45 oC_normalized (A)', fontsize=16, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.02)  # Adjust the range for the x-axis
plt.ylim(0, 0.02)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_37oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_06_12_2024/normalized_combined_fastqs_barseq_pool_counts_06_12_2024.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'KFRB023_1'
y_column = 'KFRB024_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_KFRB023_1 Abundance', fontsize=14)
plt.ylabel('Normalized_KFRB024_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances_45 oC_normalized (B)', fontsize=16, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.02)  # Adjust the range for the x-axis
plt.ylim(0, 0.02)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_37oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_06_12_2024/normalized_combined_fastqs_barseq_pool_counts_06_12_2024.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'KFRB025_1'
y_column = 'KFRB026_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_KFRB025_1 Abundance', fontsize=14)
plt.ylabel('Normalized_KFRB026_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances_50 oC_normalized (A)', fontsize=16, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.02)  # Adjust the range for the x-axis
plt.ylim(0, 0.02)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_37oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_06_12_2024/normalized_combined_fastqs_barseq_pool_counts_06_12_2024.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'KFRB027_1'
y_column = 'KFRB028_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_KFRB027_1 Abundance', fontsize=14)
plt.ylabel('Normalized_KFRB028_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances_50 oC_normalized (B)', fontsize=16, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.02)  # Adjust the range for the x-axis
plt.ylim(0, 0.02)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_37oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()

# To do statistical t-test

### To handle rows where some values are zero and convert them to NaN before performing the Mann-Whitney U test

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu

# Load the dataset
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_biorep_all_temp_normalized.csv')

# Convert all columns to numeric, coercing errors to NaN
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Replace zeros with NaN
df.replace(0, np.nan, inplace=True)

# Define the data columns for the existing and new conditions
data_37C = df[['37•C (A)', '37•C (B)']]
data_45C = df[['45•C (A)', '45•C (B)', '45•C (C)', '45•C (D)']]
data_50C = df[['50•C (A)', '50•C (B)', '50•C (C)', '50•C (D)']]
            



# List to store p-values for existing conditions
p_values_37_vs_45 = []
p_values_37_vs_50 = []

# Iterate over rows and perform Mann-Whitney U test for each
for index, row in df.iterrows():
    # Drop NaN values from the datasets
    data_37C_values = row[data_37C.columns].dropna()
    data_45C_values = row[data_45C.columns].dropna()
    
    # Perform the Mann-Whitney U test for 37C vs. 45C
    if len(data_37C_values) > 0 and len(data_45C_values) > 0:
        statistic, p_value = mannwhitneyu(data_37C_values, data_45C_values, alternative='two-sided')
        p_values_37_vs_45.append(p_value)
    else:
        p_values_37_vs_45.append(np.nan)

    # Perform the Mann-Whitney U test for 37C vs. 50C
    #data_37C_values = row[data_25C.columns].dropna()
    data_50C_values = row[data_50C.columns].dropna()

    if len(data_37C_values) > 0 and len(data_50C_values) > 0:
        statistic, p_value = mannwhitneyu(data_37C_values, data_50C_values, alternative='two-sided')
        p_values_37_vs_50.append(p_value)
    else:
        p_values_37_vs_50.append(np.nan)

# Add the p-values to the DataFrame
df['MannWhitneyU_p_value_37_vs_45'] = p_values_37_vs_45
df['MannWhitneyU_p_value_37_vs_50'] = p_values_37_vs_50

# Save the DataFrame with the calculated p-values to a new CSV file
output_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_37•C_repeated_09_30_2024/poolCount_biorep_all_temp_output_with_mannwhitneyu_p_values_zero_values_removed_combined.csv'
df.to_csv(output_path, index=False)
